# Hello World Kubeflow Pipeline

In [ ]:
from datetime import datetime
from pathlib import Path

from google.cloud import aiplatform
from google.oauth2 import service_account
from kfp.v2 import compiler, dsl
from kfp.v2.dsl import pipeline

## Config

In [ ]:
# GCP related config
PIPELINE_NAME = "hello-world-kubeflow-pipeline"
BUCKET_NAME = "my-vertex-playground-bucket"  # set up beforehand
PIPELINE_ROOT = f"gs://{BUCKET_NAME}/{PIPELINE_NAME}/"

PROJECT_ID = "open-problems-multimodal"  # set up beforehand
REGION = "us-central1"
SERVICE_ACCOUNT = (
    "vertex@open-problems-multimodal.iam.gserviceaccount.com"  # set up beforehand
)
GCP_CREDENTIALS_PATH = Path().home() / "keys" / "my-vertex-playground-key.json"

# Local path to the compiled pipeline json file
PIPELINE_JSON_PATH = "compiled_pipeline.json"

## Components

In [ ]:
@dsl.component(
    base_image="python:3.10",
)
def hello_op(
    hello_str: str,
) -> str:
    return hello_str


@dsl.component(
    base_image="python:3.10",
)
def world_op(
    hello_str: str,
) -> str:
    return hello_str + ", world!"

## Pipeline

In [ ]:
@pipeline(
    name=PIPELINE_NAME,
    pipeline_root=PIPELINE_ROOT,
)
def my_pipeline(hello_str: str):
    hello_task = hello_op(hello_str=hello_str).set_memory_limit("1G").set_cpu_limit("1")
    world_task = (
        world_op(hello_str=hello_task.output).set_memory_limit("1G").set_cpu_limit("1")
    )

# Compile the pipeline
compiler.Compiler().compile(pipeline_func=my_pipeline, package_path=PIPELINE_JSON_PATH)

## Submit job

In [ ]:
TIMESTAMP = datetime.now().strftime("%Y%m%d-%H%M%S")
JOB_NAME = f"{PIPELINE_NAME}-{TIMESTAMP}"
credentials = service_account.Credentials.from_service_account_file(
    GCP_CREDENTIALS_PATH
)
job = aiplatform.PipelineJob(
    display_name=JOB_NAME,
    credentials=credentials,
    template_path=PIPELINE_JSON_PATH,
    job_id=JOB_NAME,
    pipeline_root=PIPELINE_ROOT,
    enable_caching=True,
    project=PROJECT_ID,
    location=REGION,
    parameter_values={"hello_str": "Hello"},
)
job.submit(service_account=SERVICE_ACCOUNT)